# Câu 6: Association Rules – BAKERY (1.0 mark)

Use dataset 75000 (select suitable files in this folder) to build the model to identify sets 
of items that are frequently bought together with two cases: 
- Using: Id
- Using: Flavor and Food name (in goods.csv).

Read more information here:

http://users.csc.calpoly.edu/~dekhtyar/466-Spring2018/labs/lab2.466.pdf

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
spark = SparkSession.builder.appName('AssociationRules_BAKERY').getOrCreate()
spark

## Read dataset and Pre-processing data

Items Table. Files XXXXi.csv. Each line of the file represents a single tuple from the Items table. 
- The columns are: Receipt(number), Quantity, Item
- Example:

        1,3,7
        1,4,15
        1,2,49
        1,5,44
        (note, that the item IDs may be out of order)

In [5]:
# Read 1000i.csv file for list of transactions
# ReceiptId, Quantity, Item Id
bakery = spark.read.csv(f"75000/75000i.csv", header=False, inferSchema=True)
bakery = bakery.toDF("ReceiptId", "Quantity", "Id")

In [6]:
#2. Cho biết dữ liệu có bao nhiêu dòng, 
bakery.count()

266209

In [7]:
# in scheme. 
bakery.printSchema()

root
 |-- ReceiptId: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Id: integer (nullable = true)



In [8]:
bakery.show(5)

+---------+--------+---+
|ReceiptId|Quantity| Id|
+---------+--------+---+
|        1|       1| 21|
|        1|       5| 11|
|        2|       1|  7|
|        2|       3| 11|
|        2|       4| 37|
+---------+--------+---+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import isnan, when, count, col

In [10]:
#3. Kiểm tra dữ liệu NaN, null
bakery.select([count(when(isnan(c), c)).alias(c) for c in bakery.columns]).toPandas().T

,0
ReceiptId,0
Quantity,0
Id,0


 => Không có dữ liệu NaN

In [11]:
bakery.select([count(when(col(c).isNull(), c)).alias(c) for c in bakery.columns]).toPandas().T

,0
ReceiptId,0
Quantity,0
Id,0


 => Không có dữ liệu Null

In [12]:
#4. Kiểm tra dữ liệu trùng. 
num_rows = bakery.count()
num_dist_rows = bakery.distinct().count()
dup_rows = num_rows - num_dist_rows

In [13]:
display(num_rows, num_dist_rows, dup_rows)

266209

266209

0

 => Không có dữ liệu trùng

In [14]:
# Convert dataframe phù hợp FPGrowth
bakery_transaction = bakery.groupBy("ReceiptId").agg({"Id": "collect_list"}) \
                                 .withColumnRenamed("collect_list(Id)", "items")

In [15]:
bakery_transaction.show(5, truncate=False)

+---------+---------------+
|ReceiptId|items          |
+---------+---------------+
|1        |[21, 11]       |
|2        |[7, 11, 37, 45]|
|3        |[3, 42, 33]    |
|4        |[5, 12, 47, 17]|
|5        |[42, 6, 18]    |
+---------+---------------+
only showing top 5 rows



## Build the model to identify sets of items that are frequently bought together

### Case 1: Using Id

In [16]:
from pyspark.ml.fpm import FPGrowth

In [17]:
# Fit the FP-growth model
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.003)
model = fpGrowth.fit(bakery_transaction)

In [18]:
# Display frequent itemsets.
model.freqItemsets.show()

+--------+----+
|   items|freq|
+--------+----+
|    [19]|5685|
|[19, 27]| 359|
|[19, 33]| 334|
| [19, 1]|2764|
|[19, 28]| 408|
|[19, 37]| 274|
|[19, 35]| 312|
|[19, 16]| 286|
| [19, 4]| 388|
|[19, 46]| 324|
|[19, 15]| 298|
| [19, 5]| 323|
|[19, 22]| 368|
|[19, 32]| 297|
|[19, 45]| 344|
|[19, 47]| 331|
| [19, 3]| 294|
|[19, 14]| 350|
|[19, 11]| 296|
| [19, 0]| 305|
+--------+----+
only showing top 20 rows



In [19]:
# transform examines the input items against all the association rules and summary
# consequents as prediction
mostPopularItemInABasket = model.transform(bakery_transaction)

In [20]:
mostPopularItemInABasket.show(3, truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ReceiptId  | 1                                                                                                                                                                                   
 items      | [21, 11]                                                                                                                                                                            
 prediction | [19, 33, 27, 1, 28, 35, 16, 46, 4, 15, 22, 5, 32, 45, 47, 14, 0, 18, 49, 9, 44, 42, 7, 17, 29, 2, 40, 37, 3]                                                                        
-RECORD 1-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ReceiptId  | 2          

In [21]:
# Display generated association rules
model.associationRules.show()

+----------+----------+--------------------+------------------+--------------------+
|antecedent|consequent|          confidence|              lift|             support|
+----------+----------+--------------------+------------------+--------------------+
|      [14]|      [19]|0.055014146494812954|0.7257802967653424|0.004666666666666667|
|      [14]|      [41]|0.044011317195850365|0.6496455008244002|0.003733333333333...|
|      [14]|      [21]| 0.04306821754165357|0.9960272326931907|0.003653333333333...|
|      [14]|      [31]|0.044011317195850365|0.6540219515927833|0.003733333333333...|
|      [14]|      [38]| 0.04306821754165357|1.0148024868438636|0.003653333333333...|
|      [14]|      [33]| 0.05910091166299906|0.7188725875324244|0.005013333333333333|
|      [14]|      [17]|0.050927381326626844|0.6908217759987364|             0.00432|
|      [14]|       [1]| 0.06523105941527821|0.7801514042650081|0.005533333333333334|
|      [14]|      [24]|0.044168500471549826|0.6494094364568197|0.

In [22]:
mostPopularItemInABasket.write.parquet("MPIB.parquet", mode = "overwrite")

In [23]:
mostPopularItemInABasket.toPandas().to_csv('mostPopularItemInABasket_ID.csv', index = False)

### Case 2: Using Flavor and Food name (in goods.csv)

In [24]:
from pyspark.sql.functions import concat, col, lit, regexp_replace, concat_ws

In [25]:
# Read goods.csv file and assign column classes
goods_raw = spark.read.csv('75000/goods.csv', 
                      header=True, inferSchema=True)

In [26]:
goods_raw.show(5, truncate=False)

+---+------------+------+-----+------+
|Id |Flavor      |Food  |Price|Type  |
+---+------------+------+-----+------+
|0  |'Chocolate' |'Cake'|8.95 |'Food'|
|1  |'Lemon'     |'Cake'|8.95 |'Food'|
|2  |'Casino'    |'Cake'|15.95|'Food'|
|3  |'Opera'     |'Cake'|15.95|'Food'|
|4  |'Strawberry'|'Cake'|11.95|'Food'|
+---+------------+------+-----+------+
only showing top 5 rows



In [27]:
# Merge the columns and remove the "'" and "spaces"
goods = goods_raw.withColumn("Flavor_and_Food_name", concat(col("flavor"), lit(" "), col("food"))) \
                 .withColumn("Flavor_and_Food_name", regexp_replace(col("Flavor_and_Food_name"), "'", "")) \
                 .drop("flavor", "type", "price", "drink", "food")

In [28]:
goods.show(5, truncate=False)

+---+--------------------+
|Id |Flavor_and_Food_name|
+---+--------------------+
|0  |Chocolate Cake      |
|1  |Lemon Cake          |
|2  |Casino Cake         |
|3  |Opera Cake          |
|4  |Strawberry Cake     |
+---+--------------------+
only showing top 5 rows



In [29]:
# Merge both dataframes based on their Item_ID
bakery_merge = bakery.join(goods, "Id", "inner")

In [30]:
# Convert the merged dataframe to the required format for FPGrowth
bakery_transaction_1 = bakery_merge.groupBy("ReceiptId").agg({"Flavor_and_Food_name": "collect_list"}) \
                                 .withColumnRenamed("collect_list(Flavor_and_Food_name)", "items")

In [31]:
bakery_transaction_1.show(3, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------------------------------
 ReceiptId | 1                                                                                                       
 items     | [Ganache Cookie, Apple Pie]                                                                             
-RECORD 1------------------------------------------------------------------------------------------------------------
 ReceiptId | 6                                                                                                       
 items     | [Casino Cake, Strawberry Cake, Chocolate Croissant]                                                     
-RECORD 2------------------------------------------------------------------------------------------------------------
 ReceiptId | 12                                                                                                      
 items     | [Casino Cake, Opera Cake, Raspberry Lemonad

In [32]:
# Fit the FP-growth model
fpGrowth_1 = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.003)
model_1 = fpGrowth_1.fit(bakery_transaction_1)

In [33]:
# Display frequent itemsets.
model_1.freqItemsets.show(5, truncate=False)

+------------------------------------+----+
|items                               |freq|
+------------------------------------+----+
|[Vanilla Meringue]                  |3179|
|[Vanilla Meringue, Lemon Tart]      |252 |
|[Vanilla Meringue, Marzipan Cookie] |277 |
|[Vanilla Meringue, Cheese Croissant]|260 |
|[Vanilla Meringue, Chocolate Tart]  |233 |
+------------------------------------+----+
only showing top 5 rows



In [34]:
mostPopularItemInABasket_1 = model_1.transform(bakery_transaction_1)

In [35]:
mostPopularItemInABasket_1.show(3, truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ReceiptId  | 1                                                                                                                                                                                                                                                    

In [36]:
type(mostPopularItemInABasket_1)

pyspark.sql.dataframe.DataFrame

In [37]:
# chuyển list array thành string
from pyspark.sql.types import StringType

In [38]:
mostPopularItemInABasket_1.printSchema()

root
 |-- ReceiptId: integer (nullable = true)
 |-- items: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [39]:
mostPopularItemInABasket_1.createOrReplaceTempView("popular_items")

In [40]:
DF_cast = mostPopularItemInABasket_1.select('ReceiptId',  
 mostPopularItemInABasket_1.items.cast(StringType()),
 mostPopularItemInABasket_1.prediction.cast(StringType())) 
DF_cast.printSchema()

root
 |-- ReceiptId: integer (nullable = true)
 |-- items: string (nullable = false)
 |-- prediction: string (nullable = true)



In [41]:
DF_cast.head(3)

[Row(ReceiptId=1, items='[Ganache Cookie, Apple Pie]', prediction='[Lemon Tart, Cheese Croissant, Marzipan Cookie, Lemon Cake, Tuile Cookie, Apricot Danish, Blueberry Tart, Strawberry Cake, Chocolate Coffee, Blackberry Tart, Gongolais Cookie, Truffle Cake, Apricot Croissant, Hot Coffee, Vanilla Frappuccino, Berry Tart, Cherry Tart, Chocolate Cake, Single Espresso, Napoleon Cake, Orange Juice, Bottled Water, Coffee Eclair, Lemon Lemonade, Almond Twist, Opera Cake, Walnut Cookie, Casino Cake, Chocolate Tart]'),
 Row(ReceiptId=6, items='[Casino Cake, Strawberry Cake, Chocolate Croissant]', prediction='[Lemon Tart, Cheese Croissant, Marzipan Cookie, Chocolate Tart, Lemon Cake, Tuile Cookie, Chocolate Coffee, Blackberry Tart, Gongolais Cookie, Truffle Cake, Hot Coffee, Vanilla Frappuccino, Opera Cake, Berry Tart, Chocolate Cake, Cherry Tart, Napoleon Cake, Bottled Water, Orange Juice, Coffee Eclair, Vanilla Meringue, Green Tea, Lemon Lemonade, Almond Tart, Apple Pie, Single Espresso, Pecan 

In [42]:
DF_cast.toPandas().to_csv('mostPopularItemInABasket_String.csv', index = False)